In [5]:
import pandas as pd
import numpy as np

In [6]:
df = pd.read_csv('lending_club_loan_two.csv')

In [7]:
df['loan_repaid'] = df['loan_status'].map({'Fully Paid':1,'Charged Off':0})

In [8]:
df = df.drop('emp_title',axis=1)

In [9]:
emp_co = df[df['loan_status']=='Charged Off'].groupby('emp_length').count()['loan_status']
emp_fp = df[df['loan_status']=='Fully Paid'].groupby('emp_length').count()['loan_status']

In [10]:
df = df.drop('emp_length',axis=1)

In [11]:
df = df.drop('title',axis=1)

In [12]:
total_acc_avg = df.groupby('total_acc').mean()['mort_acc']

# %%
# filling out missing values in mort_acc
def fill_mort_acc(total_acc,mort_acc):
    if np.isnan(mort_acc):
        return total_acc_avg[total_acc]
    else:
        return mort_acc

# %%
df['mort_acc'] = df.apply(lambda x: fill_mort_acc(x['total_acc'],x['mort_acc']),axis=1)

/tmp/ipykernel_9998/1719564190.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  total_acc_avg = df.groupby('total_acc').mean()['mort_acc']


In [13]:
df = df.dropna()

In [14]:
dummies = pd.get_dummies(df[['verification_status','application_type','initial_list_status','purpose']],drop_first=True)
df = pd.concat([df.drop(['verification_status','application_type','initial_list_status','purpose'],axis=1),dummies],axis=1)

In [15]:
df['home_ownership'] = df['home_ownership'].replace(['NONE','ANY'],'OTHER')

# %%
dummies = pd.get_dummies(df['home_ownership'],drop_first=True)
df = pd.concat([df.drop(['home_ownership'],axis=1),dummies],axis=1)

In [16]:
df['address'].apply(lambda address:address[-5:])

# %%
df['zip_code'] = df['address'].apply(lambda address:address[-5:])

In [17]:
dummies = pd.get_dummies(df['zip_code'],drop_first=True)
df = pd.concat([df.drop(['zip_code'],axis=1),dummies],axis=1)

# %%
df = df.drop('address',axis=1)
df = df.drop('issue_d',axis=1)
df['earliest_cr_line'] = df['earliest_cr_line'].apply(lambda date: int(date[-4:]))
df = df.drop('grade',axis=1)
subgrade_dummies = pd.get_dummies(df['sub_grade'],drop_first=True)

# %%
df = pd.concat([df.drop('sub_grade',axis=1),subgrade_dummies],axis=1)

# %%
dummies = pd.get_dummies(df['term'],drop_first=True)
df = pd.concat([df.drop(['term'],axis=1),dummies],axis=1)

# %%
dummies = pd.get_dummies(df['loan_status'],drop_first=True)
df = pd.concat([df.drop(['loan_status'],axis=1),dummies],axis=1)

In [18]:
from sklearn.model_selection import train_test_split
X = df.drop('loan_repaid',axis=1).values
y = df['loan_repaid'].values

In [19]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=.2,random_state=101)

In [20]:
from sklearn.preprocessing import MinMaxScaler

In [21]:
scaler = MinMaxScaler()

In [22]:
X_train = scaler.fit_transform(X_train)

In [23]:
X_test = scaler.transform(X_test)

In [24]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Activation,Dropout
from tensorflow.keras.constraints import max_norm

In [25]:
model = Sequential()

2023-01-16 13:41:25.088660: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2023-01-16 13:41:25.088761: W tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:265] failed call to cuInit: UNKNOWN ERROR (303)
2023-01-16 13:41:25.088796: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (79f44b6b0317): /proc/driver/nvidia/version does not exist
2023-01-16 13:41:25.089264: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
